# Tools Installation

In [1]:
from __future__ import print_function

import pandas as pd
pd.__version__

import re
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Surprise Library

In [2]:
#Setup for surprise
!pip install scikit-surprise

     |████████████████████████████████| 6.5MB 2.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678246 sha256=4272698b9eae644dcac4460104d6caffe6a553a391f2f81d02610198cd75d431
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import GridSearchCV

# Work on Data

In [4]:
data = pd.read_csv("/content/drive/My Drive/ML_project_recommanders/Datasets/data_train.csv")
print(data.head)

<bound method NDFrame.head of                    Id  Prediction
0              r44_c1           4
1              r61_c1           3
2              r67_c1           4
3              r72_c1           3
4              r86_c1           5
...               ...         ...
1176947   r9990_c1000           4
1176948   r9992_c1000           5
1176949   r9994_c1000           3
1176950   r9997_c1000           4
1176951  r10000_c1000           3

[1176952 rows x 2 columns]>


In [0]:
#Surprise has a function that takes a dataset from a pandas dataframe.
# However, it is required that the dataframe has the form:
#3 columns [userid, movieid, rating]
#So we need to adapt the data csv given to the previous format.

In [0]:
#We have something of the form user1_movie1 rating1_1

## Reformat Data

## Surprise preparation

In [7]:
reader = Reader(rating_scale=(1, 5))

cleanedFrame = pd.DataFrame({
    'userId': data["Id"].apply(lambda x: int(re.search('r(.*)_' , x).group(1))),
    'movieId': data["Id"].apply(lambda x: int(re.search('c(.*)' , x).group(1))),
    'rating' : data["Prediction"]
})

trainingset = cleanedFrame.sample(frac = 0.7)
testset = cleanedFrame.drop(trainingset.index)

print(trainingset.head)
print(testset.head)

<bound method NDFrame.head of          userId  movieId  rating
749883     9104      617       5
834320     6922      663       4
904728     5768      706       5
416908     7585      343       3
4589       9312        4       5
...         ...      ...     ...
697575     3211      595       5
1018505    6000      805       3
801332     2613      647       4
850456     2166      672       3
126005     1424      105       1

[823866 rows x 3 columns]>
<bound method NDFrame.head of          userId  movieId  rating
7           114        1       3
12          182        1       3
13          310        1       3
28          673        1       4
34          872        1       2
...         ...      ...     ...
1176940    9958     1000       4
1176942    9961     1000       4
1176945    9970     1000       5
1176946    9976     1000       5
1176950    9997     1000       4

[353086 rows x 3 columns]>


In [0]:
reader = Reader(rating_scale=(1, 5))

#Here we call surprise function 

trainset = Dataset.load_from_df(trainingset[['userId', 'movieId', 'rating']], reader)
validationset = Dataset.load_from_df(testset[['userId', 'movieId', 'rating']], reader)

training_data = trainset.build_full_trainset()
validation_test = validationset.build_full_trainset().build_testset()

In [0]:
from surprise import KNNBaseline

In [0]:
algo1 = KNNBaseline()
sim_options = {'name': 'pearson_baseline',
               'user_based': True # compute  similarities between items
               }

bsl_options = {'method': 'als',
               'n_epochs': 5
               } 

algo1.sim_options = sim_options
algo1.bsl_options = bsl_options
algo1.k = 100

algo1.fit(training_data)

print("Now testing...")
pred_knn_items = algo1.test(validation_test)
accuracy.rmse(pred_knn_items)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Now testing...


Ancien meilleur score:
1.354, nepoch: 5, k: 1